## abridged metaprogramming classics - this episode: pytest

![](ac-lotr.jpg)
<br>
<center style="font-size: 20px; font-family: Source Code Pro Medium; font-weight: bold;">&lt;&lt;METADATA&gt;&gt; BOOK: ABRIDGED CLASSICS; AUTHOR: JOHN ATKINSON&lt;&lt;/METADATA&gt;&gt;</center>


I am afraid i need every minute to make this half hour worth while for you, but I'll be glad to listen to your questions and comments in the coffee break afterwards or whenever in the interwebs.

# let's imagine the internet is down 😱

![](internet-is-down.jpg)

* we're bored

* we have a laptop with Python3.8 installed

* we want to learn a bit about metaprogramming

# logical conclusion: re-implement pytest from scratch

## (about 0.6% of it)

# use inbuilt functionality directly and max 3 stdlib imports

![](luciano-ramalho-core-features.jpg)

<br>
<center style="font-size: 20px; font-family: Source Code Pro Medium; font-weight: bold;">&lt;&lt;METADATA&gt;&gt; TALK: BEYOND PARADIGMS: A NEW KEY TO GROK PYTHON & OTHER LANGUAGES; SPEAKER: LUCIANO RAMALHO&lt;&lt;/METADATA&gt;&gt;</center>

# minimal theoretical knowledge to follow this talk

## modules and functions are first class citizens

&nbsp;

* assignable / passable / returnable&nbsp;&nbsp;&nbsp; => &nbsp; `lobster = eggs(spam)`

* inspectable&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; => &nbsp; `dir(spam)`, `spam.__name__`

#### everything that looks like `__foo__` is part of internal Python mechanics and does interesting stuff

* mutable&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; => &nbsp; `sys.path = []`, `spam.new_attr = 42`

#### good reads from the Python docs: [Execution Model](https://docs.python.org/3/reference/executionmodel.html)  | [Data Model](https://docs.python.org/3/reference/datamodel.html)

# what is pytest?

&nbsp;

pytest is a very mature and powerful testing framework with a low entrance barrier. It does its best to make it easy to get started writing and running tests. Due to its feature set and plugin based architecture it is very modifiable and extensible.

# https://pytest.org/en/latest/talks.html

&nbsp;

##### or hop in your time machine, travel back to pycon.de 2019 and watch the introduction by @hackebrot

# metawhatchamacallit?

Let's have a quick look at the definition of metaprogramming, so that we can spot, when we are doing it.

> Metaprogramming is a programming technique in which computer programs have the ability to treat other programs as their data. It means that a program can be designed to read, generate, analyze or transform other programs, and even modify itself while running. 
>
> **[lots more]**
>
> &mdash;&mdash; [Wikipedia: Metaprogramming](https://en.wikipedia.org/wiki/Metaprogramming)

> Metaprogramming is using code to do stuff with code.
>
> &mdash;&mdash; Abridged Wikipedia (if it would exist)

# first feature:

## automatic test discovery and execution

## find and run functions with the right name in the right place

### `**/test_*.py:test_*`

# [pytest] a test suite

In [ ]:
!tree tests

##### a bunch of Python modules containing test functions in arbitrarily nested folders

# [pytest] minimal test anatomy

In [ ]:
# %load tests/basics/test_demo.py
def test_passes():
    """function is executed without error => test passes."""


def test_fails_assert():
    assert 0, "assertion fails => test fails"


def test_fails_exception():
    int("x")  # error anywhere => test fails"


def non_test_function():
    """This is not a test."""


# [pytest] only collect the tests

In [1]:
!pytest --collect-only tests/basics

tests/basics/test_demo.py::test_passes
tests/basics/test_demo.py::test_fails_assert
tests/basics/test_demo.py::test_fails_exception
tests/basics/folder/test_in_a_folder.py::test_passes_in_a_folder
tests/basics/folder/test_in_a_folder.py::test_fails_in_a_folder

no tests ran in 0.03s


# [pico_pytest] collect paths to the test modules

In [2]:
from pathlib import Path  # stdlib import number one 😱

def pico_pytest(path, collect_only=False):
    """Re-implementation of pytest - how hard can it be?"""
    paths = Path(path).glob("**/test_*.py")
    if collect_only:
        print("\n".join([str(path) for path in paths]))

pico_pytest("tests/basics", collect_only=True)

tests/basics/test_demo.py
tests/basics/folder/test_in_a_folder.py


##### (no metaprogramming in sight yet)

# next step:

## load a test module programmatically

# [fail] old skool import: hardcoded

In [ ]:
try:
    import tests.collect.test_demo
except ImportError as e:
    print(f"{e=}")

##### (because it's not in a package (no `__init__.py`))

# [success] old skool import: hardcoded

In [3]:
import sys  # stdlib import number two 😱
sys.path[:2]

['/talk', '/usr/local/lib/python38.zip']

In [ ]:
sys.path.append("tests/basics")
import test_demo
test_demo

# [stdlib] new skool import: programmatically

In [ ]:
import importlib  # stdlib import number three 😱
help(importlib)

# let's not use a library - we can use a builtin instead

In [ ]:
del importlib

# new skool import: programmatically

In [4]:
print(__import__.__doc__[:266])

__import__(name, globals=None, locals=None, fromlist=(), level=0) -> module

Import a module. Because this function is meant for use by the Python
interpreter and not for general use, it is better to use
importlib.import_module() to programmatically import a module.


In [ ]:
test_module_from_function = __import__("test_demo")

In [ ]:
test_demo is test_module_from_function  # modules are singleton objects

# 🍬 `import foo` : syntax sugar for 🍬

# 🍬 `foo = __import__("foo")` 🍬

# [pico_pytest] instantiate module objects from paths

In [ ]:
def pico_pytest(path, collect_only=False):
    paths = Path(path).glob("**/test_*.py")
    modules = []
    for path in paths:
        sys.path.append(str(path.parent))
        modules.append(__import__(path.stem))
    if collect_only:
        print("\n".join([str(module) for module in modules]))

pico_pytest("tests/basics", collect_only=True)

# 🌟 das metaprogrammometer 🌟 

![](metaprogrammometer.jpg)
<br>
<center style="font-size: 20px; font-family: Source Code Pro Medium; font-weight: bold;">&lt;&lt;METADATA&gt;&gt;John Cohen with IBMblr Play Machine&lt;&lt;/METADATA&gt;&gt;</center>

* filled in through this input 
* gently heated in these gals tubs
* purified in the pop corn pool
* broken up into logical units by this axe
* analyzed using the global knowledge about programming
* producing an assessment of the metaprogramming techniques and a measurement of the level of metaprogramyness

# 🌟 das metaprogrammometer readings 🌟 

## altering the program itself by loading arbitrary executable code

## (modules containing test functions)

## and adding them to a data structure for later use

# 🌟

# [pico_pytest] pull test module import into a function

In [ ]:
def import_module(path):
    sys.path.append(str(path.parent))
    module = __import__(path.stem)
    sys.path.pop()  # can't hurt to tidy up after yourself ...
    return module

In [ ]:
def pico_pytest(path, collect_only=False):
    paths = Path(path).glob("**/test_*.py")
    modules = [import_module(path) for path in paths]
    if collect_only:
        print("\n".join([str(module) for module in modules]))

pico_pytest("tests/basics", collect_only=True)

# next step:

## collecting tests from a module

# the namespace of a module is a `name:object` mapping

#  it is accessible via `__dict__` (and behaves like one)

In [ ]:
test_demo.test_passes              # hardcoded old skool attribute access

In [ ]:
test_demo.__dict__['test_passes']  # access via namespace mapping

In [ ]:
# in case the "."-key is broken on your keyboard
getattr(test_demo, "__dict__")["test_passes"]

# 🍬 access to attributes via dot notation is syntax sugar 🍬

# [pico_pytest] a function to collect test functions

In [ ]:
from types import FunctionType  # third stdlib import 😱

def collect_test_functions(module):
    functions = []
    for name, obj in test_demo.__dict__.items():
        if (
            isinstance(obj, FunctionType) 
            and name.startswith("test_")
        ):
            functions.append(obj)
    return functions

collect_test_functions(test_demo)

# let's not use that third import yet

In [ ]:
del FunctionType

# [pico_pytest] a function to collect test functions II

In [ ]:
def collect_test_functions(module):
    return [
        obj
        for obj in module.__dict__.values() if
            obj.__class__.__name__ == "function" 
            and obj.__name__.startswith("test_")
    ]

collect_test_functions(test_demo)

# [pico_pytest] put all tests from all modules in a list

In [ ]:
def pico_pytest(path, collect_only=False):
    paths = Path(path).glob("**/test_*.py")
    modules = [import_module(path) for path in paths]
    tests = []
    for module in modules:
        tests.extend(collect_test_functions(module))
    if collect_only:
        print("\n".join([str(test) for test in tests]))

pico_pytest("tests/basics", collect_only=True)

# next step:

## executing the tests

# [pico_pytest] a function to execute a test function

In [ ]:
def execute_test(f):
    result = "."
    try:
        f()
        return "passed"
    except Exception as e:
        result = "F"
        return e
    finally:
        print(result, end="")
        
execute_test(test_demo.test_fails_assert)

# [pico_pytest] call all the tests and report on the results

In [ ]:
def pico_pytest(path, collect_only=False):
    paths = Path(path).glob("**/test_*.py")
    modules = [import_module(path) for path in paths]
    tests = []
    for module in modules:
        tests.extend(collect_test_functions(module))
    if collect_only:
        print("\n".join([str(test) for test in tests]))
    else:
        results = {test.__name__: execute_test(test) for test in tests}
        print("\n" + "\n".join([f"{n}: {r!r}" for n, r in results.items()]))

pico_pytest

# [pytest] running `tests/basics`

In [ ]:
!pytest tests/basics

# [pico_pytest] running `tests/basics`

In [ ]:
pico_pytest("tests/basics")

# 🌟 das metaprogrammometer readings 🌟 

## run programmatically collected functions

## from programmatically imported modules

# 🌟

# second freature:

## automatic dependency injection (`fixture` decorator)

In [ ]:
# %load tests/fixtures/test_fixtures.py
import pytest


@pytest.fixture  # think of this as registering a fixture function
def the_answer():
    return 42


def test_using_fixture_passes(the_answer):  # request fixture result via name
    assert the_answer == 42


def test_using_fixture_fails(the_answer):
    assert the_answer == 23, f"{the_answer} is not 23 :("


&nbsp;

* test functions can request `fixtures` via their the parameter list

* `fixtures` are specially marked functions that get executed by pytest

* their results are injected into the requesting function when executed

* the connection is made through the name of the function

# next step:

## collect fixture functions from the test modules

# transfer fixture function objects to a map during import

In [ ]:
name2fixture = {}  # dict lives as long as the process (test session) lives

In [ ]:
def fixture(f):
    """pico_pytest decorator to register a function as fixture."""
    name2fixture[f.__name__] = f
    print(f"<fixture({f.__name__}) => registered>", file=sys.stderr)
    
fixture

# test transfer directly

In [ ]:
def syntax_sugar_free():
    pass

syntax_sugar_free = fixture(syntax_sugar_free)

In [ ]:
print(syntax_sugar_free)

In [ ]:
name2fixture

# 🍬 `@`:  decorator syntax sugar [(PEP 318)](https://www.python.org/dev/peps/pep-0318/) 🍬

#### pass a function to a function and assign the result to the name of the original function

In [ ]:
@fixture
def with_syntax_sugar():
    pass

with_syntax_sugar is None

In [ ]:
name2fixture

# test fixture registration from a test module

In [ ]:
import pytest

pytest.fixture = fixture
print(f"{pytest.fixture.__doc__}")

# 🌟 das metaprogrammometer readings 🌟 

## replacing library code in a running program

## (a.k.a monkeypatching)

# 🌟 🌟

# module import triggers fixture registration now

In [ ]:
test_fixtures = import_module(Path("tests/fixtures/test_fixtures.py"))

In [ ]:
name2fixture

# next step:

## figure out if a test requested a fixture

In [ ]:
def test_function(parameter1, parameter2):  # <= we need those parameter names
    nameInFunction = 1
    
dir(test_function)[:5]

In [ ]:
print(dir(test_function.__code__)[-17:-1])

In [ ]:
print(test_function.__code__.co_argcount, test_function.__code__.co_varnames)

# [pico_pytest] add automatic dependency injection

In [ ]:
def execute_test(f):
    params = f.__code__.co_varnames[:f.__code__.co_argcount]
    kwargs = {n: fixture() for n, fixture in name2fixture.items() if n in params}
    result = "."
    try:
        f(**kwargs)  # **mapping: call function with key=value keyword arguments
        return "passed"
    except Exception as e:
        result = "F"
        return e
    finally:
        print(result, end="")

pico_pytest("tests/fixtures")  # no change necessary in pico_pytest function

# 🌟 das metaprogrammometer readings 🌟 

## inspecting attributes of callable objects from a mapping,

## selectively calling them to inject their results

## into another programmatically called function

# 🌟 🌟

# third feature:

## marking tests with arbitrary names and 

## selecting subsets of tests via boolean expression

# [pytest] selecting tests using an expression

In [ ]:
!pytest --collect-only -m "lucy and charlie" tests

# the marked tests we just selected from

In [ ]:
# %load tests/marking/test_marking.py
import pytest

@pytest.mark.charlie                # marker name can be any valid identifier
def test_fails_marked_charlie():
    assert 0, "This is bad!"

@pytest.mark.lucy
def test_passes_marked_lucy():
    pass

@pytest.mark.lucy
@pytest.mark.charlie                # a test can be marked with several markers
def test_passes_marked_lucy_and_charlie():
    pass


# next step:

# maintain a list of marks on a function object

# problem:

## how do we create a function `mark` usable as a decorator

## that gets an arbitrary name passed via dot notation?

### let's work our way slowly towards this

# `attach_mark`: a function to maintain a list of marks

In [ ]:
def attach_mark(f, m):
    try:
        f.pico_pytest_marks.append(m)
    except AttributeError:
        f.pico_pytest_marks = [m]
        # ^^^^^^^^^^^^^^^^^ => attach a brand new function attribute
    return f

In [ ]:
def spam():
    pass

spam = attach_mark(spam, "patty")
print(spam.pico_pytest_marks)

# I can't pass the mark in as a second argument like that

# so there has to be another way

## name lookup in Python: LEGB (local, enclosing, global, builtin)

In [ ]:
def attach_mark(f):       # we can't pass the name as argument here ...
    try:
        f.pico_pytest_marks.append(m)   # access m from outside
    except AttributeError:
        f.pico_pytest_marks = [m]
    return f

m = "lucy"                # we can set it in another accessible scope
spam = attach_mark(spam)
m = "franklin"            # and change as needed
spam = attach_mark(spam)
spam.pico_pytest_marks

# next step:

# use a "portable" scope by creating a closure

# factory to create decorators carrying their context

In [ ]:
def mark_attacher_factory(m):  # function creates a function with enclosed m
    def attach_mark(f):
        try:
            f.pico_pytest_marks.append(m)  # access m from enclosing scope
        except AttributeError:
            f.pico_pytest_marks = [m]
        print(f"<attach_mark({f.__name__}) => '{m}'>", file=sys.stderr)
        return f
    return attach_mark

mark_attacher_factory("schroeder")

# trying that out directly

In [ ]:
spam = mark_attacher_factory("schroeder")(spam)
print(spam.pico_pytest_marks)

In [ ]:
@mark_attacher_factory("snoopy")
def spam():  # this assigns a new function object to the name "spam"
    pass

print(spam.pico_pytest_marks)

# let's visualize this (using pythontutor.com)

## [sugar free](http://pythontutor.com/visualize.html#code=def%20mark_attacher_factory%28m%29%3A%0A%20%20%20%20def%20attach_mark%28f%29%3A%0A%20%20%20%20%20%20%20%20if%20hasattr%28f,%20%22pico_pytest_marks%22%29%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20f.pico_pytest_marks.append%28m%29%0A%20%20%20%20%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20f.pico_pytest_marks%20%3D%20%5Bm%5D%0A%20%20%20%20%20%20%20%20print%28f%22%3Cattach_mark%28%7Bf.__name__%7D%29%20%3D%3E%20'%7Bm%7D'%3E%22%29%0A%20%20%20%20%20%20%20%20return%20f%0A%20%20%20%20return%20attach_mark%0A%0Adef%20spam%28%29%3A%0A%20%20%20%20pass%0A%0Aspam%20%3D%20mark_attacher_factory%28%22schroeder%22%29%28spam%29%0Aprint%28spam.pico_pytest_marks%29&cumulative=true&curInstr=0&heapPrimitives=true&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

## [with syntax sugar](http://pythontutor.com/visualize.html#code=def%20mark_attacher_factory%28m%29%3A%0A%20%20%20%20def%20attach_mark%28f%29%3A%0A%20%20%20%20%20%20%20%20if%20hasattr%28f,%20%22pico_pytest_marks%22%29%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20f.pico_pytest_marks.append%28m%29%0A%20%20%20%20%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20f.pico_pytest_marks%20%3D%20%5Bm%5D%0A%20%20%20%20%20%20%20%20print%28f%22%3Cattach_mark%28%7Bf.__name__%7D%29%20%3D%3E%20'%7Bm%7D'%3E%22%29%0A%20%20%20%20%20%20%20%20return%20f%0A%20%20%20%20return%20attach_mark%0A%0A%40mark_attacher_factory%28%22snoopy%22%29%0Adef%20spam%28%29%3A%20%20%23%20this%20assigns%20a%20new%20function%20object%20to%20the%20name%20%22spam%22%0A%20%20%20%20pass%0A%0Aprint%28spam.pico_pytest_marks%29&cumulative=true&curInstr=0&heapPrimitives=true&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

# what does it look like again in pytest?

In [ ]:
import pytest

@pytest.mark.snoopy  # this is what we want to achieve ...
def test_spam():
    pass

test_spam.pytestmark

# next step:

## hook into the dot notation of `mark`

# hook into attribute access via `__getattr__`

In [ ]:
class MarkAttacherFactoryFactory:
    """pico_pytest factory factory to attach marks to tests."""
    def __getattr__(self, m):  # <- called when normal attribute lookup fails
        return mark_attacher_factory(m)

mark = MarkAttacherFactoryFactory()

# 🍬 dot notation for all attribute access is syntax sugar 🍬

# try `mark` with modified attribute access

In [ ]:
spam = mark.woodstock(spam)
print(spam.pico_pytest_marks)

In [ ]:
@mark.peggy
def spam():
    pass

print(spam.pico_pytest_marks)

# time to test it with a test module

In [ ]:
pytest.mark = mark
pytest.mark.__doc__

In [ ]:
test_marking = import_module(Path("tests/marking/test_marking.py"))

# looking at the marked tests

In [ ]:
tests_marking = collect_test_functions(test_marking)

In [ ]:
for test in tests_marking:
    print(f"{test.__name__=:<42} {getattr(test, 'pico_pytest_marks')}")
    #                     ^^^ emoji driven f-string formatting 😄

# 🌟 das metaprogrammometer readings 🌟 

## closure creation on the fly and 

## overriding attribute access using a language protocol

# 🌟 🌟

# next step:

## selecting marked test with a boolean expression

## using the mark names

# [pytest mark recap] selecting some tests

In [ ]:
!pytest --collect-only -m "lucy or charlie" tests

# code generation and evaluation

In [ ]:
OPERATORS = ["and", "or", "not"]

def evaluate(expression, names):
    tokens = []
    for token in expression.split():
        if token in names:
            tokens.append("True")
        elif token in OPERATORS:
            tokens.append(token)
        else:
            tokens.append("False")
    return eval(" ".join(tokens))

evaluate("spam or eggs", ["eggs"])

# trying `evaluate` on a few examples

In [ ]:
for expression, marks, expectation in [                    # <generated code>
    ("lucy or charlie",      [],                  False),  # "False or False"
    ("lucy or charlie",      ["lucy"],            True),   # "True or False"
    ("lucy and charlie",     ["lucy"],            False),  # "True and False"
    ("lucy and not charlie", ["lucy", "charlie"], False),  # "True and not True"
]:
    result = evaluate(expression, marks)
    print(f"{expression:<20} | {str(marks):20} => {result is expectation=}")

# [pico_pytest] a function to filter the test functions

In [ ]:
def filter_tests(tests, expression):
     return [
         test for test in tests 
         if evaluate(expression, getattr(test, "pico_pytest_marks", []))
     ]

filter_tests

# 🌟 das metaprogrammometer readings 🌟 

## dynamic code generation and execution 

## in order to decide if programmatically loaded test functions

## from programmatically imported modules should be executed

# 🌟 🌟 🌟 🤯

# [pico_pytest] add some info about deselected tests

In [ ]:
def filter_tests(tests, expression):
    remaining = [
        test for test in tests 
        if evaluate(expression, getattr(test, "pico_pytest_marks", []))
    ]
    if deselected := len(tests) - len(remaining):     # PEP 572
        print(
            f"<filter_tests(...) => {deselected=}>",  # bpo-36817
            file=sys.stderr
        )
    return remaining

filter_tests

# try filtering on the test module with marked tests

In [ ]:
filter_tests(tests_marking, "lucy or charlie")

In [ ]:
filter_tests(tests_marking, "lucy")

In [ ]:
filter_tests(tests_marking, "lucy and not charlie")

# pico_pytest: code complete 😉

In [ ]:
def pico_pytest(path, collect_only=False, mark_expression=None):
    paths = Path(path).glob("**/test_*.py")
    modules = [import_module(path) for path in paths]
    tests = []
    for module in modules:
        tests.extend(collect_test_functions(module))
    if mark_expression:
        tests = filter_tests(tests, mark_expression)
    if collect_only:
        print("\n".join([str(test) for test in tests]))
    else:
        results = {test.__name__: execute_test(test) for test in tests}
        print("\n" + "\n".join([f"{n}: {r!r}" for n, r in results.items()]))

# [quick refactoring] one import left ...

In [ ]:
from itertools import chain  # one import left ...

def pico_pytest(path, collect_only=False, mark_expression=None):
    paths = Path(path).glob("**/test_*.py")
    modules = [import_module(path) for path in paths]
    tests = list(chain(*[collect_test_functions(module) for module in modules]))
    if mark_expression:
        tests = filter_tests(tests, mark_expression)
    if collect_only:
        print("\n".join([str(test) for test in tests]))
    else:
        results = {test.__name__: execute_test(test) for test in tests}
        print("\n" + "\n".join([f"{n}: {r!r}" for n, r in results.items()]))

# acceptance tests

# [pico_pytest] collecting tests

In [ ]:
pico_pytest("tests", collect_only=True)

# [pico_pytest] selecting tests

In [ ]:
pico_pytest("tests", collect_only=True, mark_expression="lucy")

# [pico_pytest] running tests

In [ ]:
pico_pytest("tests", mark_expression="not lucy and not charlie")

# that's it folks - if you are still here ...

![](awesome-lotr.jpg)

# thank you!

### `<<<=ABRIDGED METAPROGRAMMING CLASSICS - METADATA=>>>`

#### `[MATERIALS]=> https://gitlab.com/obestwalter/pico-pytest`

#### `[SPEAKER]=> https://oliver.bestwalter.de`

#### `<<<=[EMPLOYER]=>>>`
![](avira.png)

##### `Avira loves Python and is hiring - come join us!`